In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *

from qiskit_aer import Aer

We can simulate without using `qiskit` just by using python packages. There we don't need to worry about the initial state $\rho$. We assume it is given to us, in classical matrix form. So the expectation values can be calculated just by calculating traces.

# Notes
Let, $\rho = \sum_{i=1}^d c_i X_i$

It is assumed that $X_i$'s are independent and orthonormal i.e $Tr(X_i^\dagger X_j) = \delta_{ij}$.

Then, $Tr(\rho X_i) = c_i$

Our initial state is $\rho = \rho_1 \oplus \rho_2 \oplus \rho_3$, where,

$$\rho_1 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & 1 \\
    1 & 1 & 1
\end{bmatrix}
            \quad 
\rho_2 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & -1 \\
    1 & -1 & 1
\end{bmatrix} \quad
\rho_3 = \frac{1}{2}
\begin{bmatrix}
    1 & 0 \\ 0 & 1
\end{bmatrix}$$


We want to find `Pauli Decomposition` of $\rho$ i.e. to find $c_i$'s where $X_i$'s are Pauli matrices/tensor products of Pauli matrices.

# Reference from [here](https://docs.quantum.ibm.com/build/specify-observables-pauli)

In [2]:
#TODO: initialize a mixed state
from scipy.linalg import block_diag
from qiskit.quantum_info import DensityMatrix


# specify the quantum state in an array
rho_1 = (1/3)*np.array([[1,1,1],
                        [1,1,1],
                        [1,1,1]])

rho_2 = (1/3)*np.array([[1,1,1],
                        [1,1,-1],
                        [1,-1,1]])

rho_3 = (1/2)*np.array([[1,0],
                        [0,1]])

# direct sum of rho_1, rho_2, and rho_3

rho =  block_diag(rho_1, rho_2, rho_3)
# this is our initial mixed state

#convert rho into density matrix
rho = DensityMatrix(rho)

In [3]:
DensityMatrix.is_valid(rho)

False

In [4]:
# Pauli decomposition
from qiskit.quantum_info import SparsePauliOp

pauli_decomposition = SparsePauliOp.from_operator(rho)

In [5]:
# pauli decomposition
paulis = pauli_decomposition.to_list()

paulis

[('III', (0.375+0j)),
 ('IXI', (0.08333333333333333+0j)),
 ('IXX', (0.08333333333333333+0j)),
 ('IXZ', (0.08333333333333333+0j)),
 ('IYY', (0.08333333333333333-0j)),
 ('IZI', (-0.04166666666666666+0j)),
 ('XXI', (0.08333333333333333+0j)),
 ('XXX', (0.08333333333333333+0j)),
 ('XXZ', (-0.08333333333333333+0j)),
 ('XYY', (-0.08333333333333333+0j)),
 ('YXY', (0.08333333333333333-0j)),
 ('YYI', (0.08333333333333333-0j)),
 ('YYX', (0.08333333333333333-0j)),
 ('YYZ', (-0.08333333333333333+0j)),
 ('ZII', (-0.04166666666666667+0j)),
 ('ZIX', (0.16666666666666666+0j)),
 ('ZXI', (0.08333333333333333+0j)),
 ('ZXX', (0.08333333333333333+0j)),
 ('ZXZ', (0.08333333333333333+0j)),
 ('ZYY', (0.08333333333333333-0j)),
 ('ZZI', (0.04166666666666667+0j)),
 ('ZZX', (0.16666666666666666+0j))]

## Here though we have decomposed our state $\rho$ into Pauli matrices, we don't know how to initialize our circuit in this particular state 🥲

# Let us also try implementing this _from scratch_ without using `qiskit`.

In [6]:
import numpy as np
import sympy as sp

In [7]:
import numpy as np

# check if a given matrix is a density matrix or not
def is_density_matrix(matrix):
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        print("Matrix is not square")
        return False
    
    # Check if the matrix is Hermitian
    if not np.allclose(matrix, matrix.conj().T):
        print("Matrix is not Hermitian")
        return False
    
    # Check if the trace of the matrix is 1
    if not np.isclose(np.trace(matrix), 1):
        print("Trace of the matrix is not 1")
        return False
    
    # Check if all eigenvalues are non-negative
    eigenvalues = np.linalg.eig(matrix)[0]
    if not np.all(eigenvalues >= -1e-10):
        print("Not all eigenvalues are non-negative")
        return False
    
    return True

In [8]:
# # generating a random density matrix from scratch

# # Generate a random Hermitian matrix
# random_matrix = np.random.rand(2, 2) + 1j * np.random.rand(2, 2)
# hermitian_matrix = (random_matrix + random_matrix.conj().T) / 2

# # Normalize the matrix to have trace 1
# trace = np.trace(hermitian_matrix)
# density_matrix = hermitian_matrix / trace

# print(density_matrix)   
# is_density_matrix(density_matrix)

# # not robust. prone to falure because the generated matrix may not be positive semi-definite.
# commenting it. Not useful

In [9]:
# generating density matrix using qutip
from qutip import *
dm = rand_dm(5)
print(dm)
is_density_matrix(dm.full())
  

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.34266269+3.14417851e-20j -0.00035096+7.85232818e-02j
  -0.12159281+6.48394128e-02j  0.00660583-3.24259272e-02j
   0.01847817-4.34551477e-02j]
 [-0.00035096-7.85232818e-02j  0.13575688+1.24566779e-20j
   0.0639425 -5.79869821e-02j -0.01048495-8.66735294e-03j
  -0.0034058 -6.88278728e-02j]
 [-0.12159281-6.48394128e-02j  0.0639425 +5.79869821e-02j
   0.18374463+1.68599009e-20j -0.04026533+8.40284710e-02j
   0.02637491-4.21023464e-02j]
 [ 0.00660583+3.24259272e-02j -0.01048495+8.66735294e-03j
  -0.04026533-8.40284710e-02j  0.19820619+1.81868547e-20j
  -0.01802815+5.69541991e-02j]
 [ 0.01847817+4.34551477e-02j -0.0034058 +6.88278728e-02j
   0.02637491+4.21023464e-02j -0.01802815-5.69541991e-02j
   0.13962961-7.89452186e-20j]]


True

In [10]:
I, X, Y, Z = qeye(2), sigmax(), sigmay(), sigmaz()
I, X, Y, Z = np.array(I), np.array(X), np.array(Y), np.array(Z)

$\rho = \frac{1}{3} \rho_1 \oplus \rho_2 \oplus \rho_3$, where,

$$\rho_1 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & 1 \\
    1 & 1 & 1
\end{bmatrix}
            \quad 
\rho_2 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & -1 \\
    1 & -1 & 1
\end{bmatrix} \quad
\rho_3 = \frac{1}{2}
\begin{bmatrix}
    1 & 0 \\ 0 & 1
\end{bmatrix}$$

One comment to add is that, $\rho_2$ is not a density matrix; not positive semidefinite.

In [18]:
# generating density matrix rho = (1/3) rho_1 ⊕ (1/3) rho_2 ⊕ (1/3) rho_3
rho_1 = np.array([[1, 1, 1],
                [1, 1, 1],
                [1, 1, 1]]) / 3

rho_2 = np.array([[1, 1, 1],
                [1, 1, -1],
                [1, -1, 1]]) / 3

rho_3 = np.array([[1, 0],
                [0, 1]]) / 2

# direct sum of rho_1, rho_2, and rho_3
rho = np.array(block_diag(rho_1, rho_2, rho_3))/3       # it is an numpy array

# convert rho into a qutip object
rho = Qobj(rho)

rho

Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.11111111  0.11111111  0.11111111  0.          0.          0.
   0.          0.        ]
 [ 0.11111111  0.11111111  0.11111111  0.          0.          0.
   0.          0.        ]
 [ 0.11111111  0.11111111  0.11111111  0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.11111111  0.11111111  0.11111111
   0.          0.        ]
 [ 0.          0.          0.          0.11111111  0.11111111 -0.11111111
   0.          0.        ]
 [ 0.          0.          0.          0.11111111 -0.11111111  0.11111111
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.16666667  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.16666667]]